In [ ]:
import pandas as pd
from logging import getLogger


In [3]:
logger = getLogger(__name__)

In [4]:
product_df = pd.read_csv('../data/processed/openfoodfacts.csv')  # twój DataFrame z produktami

C:\Users\Michał Żądełek\AppData\Local\Temp\ipykernel_3968\3742008372.py:1: DtypeWarning: Columns (0,11,17,31,32,33,35,51,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  product_df = pd.read_csv('../data/processed/openfoodfacts.csv')  # twój DataFrame z produktami


In [8]:
product_df = product_df[product_df["completeness"] >= 0.75].reset_index(drop=True)

In [11]:
product_df

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,...,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
0,1,http://world-en.openfoodfacts.org/product/0000...,inf,1634745456,2021-10-20T15:57:36Z,1730400880,2024-10-31T18:54:40Z,lowtower1974,1.730401e+09,2024-10-31T18:54:40Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,http://world-en.openfoodfacts.org/product/0000...,elcoco,1560176426,2019-06-10T14:20:26Z,1730316925,2024-10-30T19:35:25Z,foodvisor,1.730317e+09,2024-10-30T19:35:25Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,http://world-en.openfoodfacts.org/product/0000...,touchette,1605337720,2020-11-14T07:08:40Z,1729432954,2024-10-20T14:02:34Z,roboto-app,1.729433e+09,2024-10-20T14:02:34Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,http://world-en.openfoodfacts.org/product/0000...,rigione,1541625386,2018-11-07T21:16:26Z,1729267932,2024-10-18T16:12:12Z,maldan,1.729847e+09,2024-10-25T09:08:09Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,http://world-en.openfoodfacts.org/product/0000...,jeanbono,1476947941,2016-10-20T07:19:01Z,1728034727,2024-10-04T09:38:47Z,fix-code-bot,1.729852e+09,2024-10-25T10:25:56Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293306,9991111111154,http://world-en.openfoodfacts.org/product/9991...,kiliweb,1531473835,2018-07-13T09:23:55Z,1644560987,2022-02-11T06:29:47Z,packbot,1.729959e+09,2024-10-26T16:10:45Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293307,9999091865142,http://world-en.openfoodfacts.org/product/9999...,hunsly,1540134604,2018-10-21T15:10:04Z,1644578858,2022-02-11T11:27:38Z,packbot,1.707604e+09,2024-02-10T22:29:36Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293308,9999941860884,http://world-en.openfoodfacts.org/product/9999...,kiliweb,1674984563,2023-01-29T09:29:23Z,1680002660,2023-03-28T11:24:20Z,itsjustruby,1.729930e+09,2024-10-26T08:13:54Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293309,9999991042704,http://world-en.openfoodfacts.org/product/9999...,kiliweb,1525862784,2018-05-09T10:46:24Z,1718572213,2024-06-16T21:10:13Z,geodata,1.729873e+09,2024-10-25T16:12:01Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
product_df.to_csv('../data/processed/openfoodfacts2.csv', index=False)